In [ ]:
import pickle
from checklist.editor import Editor
from checklist.pred_wrapper import PredictorWrapper
from checklist.test_suite import TestSuite
from checklist.test_types import MFT
from checklist.test_types import INV
from checklist.perturb import Perturb
import numpy as np
import pandas as pd

In [ ]:
import re
import unicodedata
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords

In [ ]:
def basic_clean(text):
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english')
    text = (unicodedata.normalize('NFKD', text)
        .encode('ascii', 'ignore')
        .decode('utf-8', 'ignore')
        .lower())
    words = re.sub(r'[^\w\s]', '', text).split()
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

# EDA
Before we can do any kind of checklisting, we need to know what kind of nouns we have in the dataset for our vocabulary, as well as which sentences are most common and get a general understanding of the dataset

## Beauty

In [ ]:
df_beauty = pd.read_csv("data\\beauty_reviews(subset).csv")

In [ ]:
words_beauty = basic_clean(''.join(str(df_beauty["review_body"].tolist())))

### Top 20 unigrams

In [ ]:
(pd.Series(nltk.ngrams(words_beauty, 1)).value_counts())[:20]

### Top 20 bigrams

In [ ]:
(pd.Series(nltk.ngrams(words_beauty, 2)).value_counts())[:20]

### Top 20 trigrams

In [ ]:
(pd.Series(nltk.ngrams(words_beauty, 3)).value_counts())[:20]

# Checklisting

## Loading models

In [ ]:
with open("baseline_model_beauty.pickle", "rb") as file:
    model_beauty = pickle.load(file)

In [ ]:
with open("baseline_model_sports.pickle", "rb") as file:
    model_sports = pickle.load(file)

In [ ]:
def predict_proba_beauty(inputs):
    p = np.array([model_beauty.predict_proba([x]) for x in inputs]).reshape(-1, 2)
    
    return p
wrapped_pp_beauty = PredictorWrapper.wrap_softmax(predict_proba_beauty)

In [ ]:
def predict_proba_sports(inputs):
    p = np.array([model_sports.predict_proba([x]) for x in inputs]).reshape(-1, 2)
    
    return p
wrapped_pp_sports = PredictorWrapper.wrap_softmax(predict_proba_sports)

## Beauty

## Capability: Vocabulary

In [ ]:
editor = Editor()
suite = TestSuite()

In [ ]:
nouns = ['product','wig','lipstick','cream','oil']
editor.add_lexicon('nouns', nouns)
pos_adj = ['amazing', 'incredible', 'extraordinary', 'excellent', 'awesome', 'nice']
neg_adj = ['terrible', 'eerie', 'odd', 'awful', 'ugly', 'strange']
editor.add_lexicon('pos_adj', pos_adj, overwrite=True)
editor.add_lexicon('neg_adj', neg_adj, overwrite=True )

In [ ]:
pos_verb_present = ['like', 'enjoy', 'appreciate', 'love',  'recommend', 'admire', 'value', 'welcome']
neg_verb_present = ['hate', 'dislike', 'regret',  'abhor', 'dread', 'despise']
pos_verb_past = ['liked', 'enjoyed', 'appreciated', 'loved', 'admired', 'valued', 'welcomed']
neg_verb_past = ['hated', 'disliked', 'regretted',  'abhorred', 'dreaded', 'despised']

editor.add_lexicon('pos_verb_present', pos_verb_present, overwrite=True)
editor.add_lexicon('neg_verb_present', neg_verb_present, overwrite=True)
editor.add_lexicon('pos_verb_past', pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb_past', neg_verb_past, overwrite=True)
editor.add_lexicon('pos_verb', pos_verb_present+ pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb', neg_verb_present + neg_verb_past, overwrite=True)

In [ ]:
# Testing individual positive words
test = MFT(pos_adj + pos_verb_present + pos_verb_past, labels=1) #1 == positive
suite.add(test, 'single positive words', 'Vocabulary', '')
# Testing individual negative words
test = MFT(neg_adj + neg_verb_present + neg_verb_past, labels=0)
suite.add(test, 'single negative words', 'Vocabulary', '')

In [ ]:
# Testing words in context
t = editor.template('{it} {nouns} {be} {pos_adj}.', it=['The', 'This', 'That'], be=['is', 'was'], labels=1, save=True)
t += editor.template('{it} {be} {a:pos_adj} {nouns}.', it=['It', 'This', 'That'], be=['is', 'was'], labels=1, save=True)
t += editor.template('{i} {pos_verb} {the} {nouns}.', i=['I', 'We'], the=['this', 'that', 'the'], labels=1, save=True)
t += editor.template('{it} {nouns} {be} {neg_adj}.', it=['That', 'This', 'The'], be=['is', 'was'], labels=0, save=True)
t += editor.template('{it} {be} {a:neg_adj} {nouns}.', it=['It', 'This', 'That'], be=['is', 'was'], labels=0, save=True)
t += editor.template('{i} {neg_verb} {the} {nouns}.', i=['I', 'We'], the=['this', 'that', 'the'], labels=0, save=True)

test = MFT(**t)
suite.add(test, 'Sentiment-laden words in context', 'Vocabulary', 
          'Use positive and negative verbs and adjectives with beauty product nouns such as product, cream, wig, etc. E.g. "This was a bad cream"')

In [ ]:
suite.run(wrapped_pp_beauty, overwrite=True)

In [ ]:
suite.summary()

## Capability: Negation

In [ ]:
editor = Editor()
suite = TestSuite()

In [ ]:
pos = ['good', 'enjoyable', 'exciting', 'excellent', 'amazing', 'great', 'engaging']
neg = ['bad', 'terrible', 'awful', 'horrible']
nouns = ['product','wig','lipstick','cream','oil',"perfume","perspirant"]


In [ ]:
editor.add_lexicon("pos",pos,overwrite=True)
editor.add_lexicon("neg",neg,overwrite=True)
editor.add_lexicon('nouns', nouns,overwrite=True)

In [ ]:
ret = editor.template('This is not {a:pos} {nouns}.', labels=0, save=True)
ret += editor.template('This is not {a:neg} {nouns}.',  labels=1, save=True)

test = MFT(**ret, name='Simple negation',
           capability='Negation', description='Very simple negations.')
    

In [ ]:
test.run(wrapped_pp_beauty, overwrite=True)

In [ ]:
test.summary()

In [ ]:
negation_df_beauty = pd.DataFrame({"text":dict(ret)["data"],"sentiment":dict(ret)["labels"]})

## Capability: Invariance

In [ ]:
def change_product(x,meta=False, *args, **kwargs):
    product = ['product','wig','lipstick','cream','oil',"perfume","perspirant"]
    ret = []
    ret_meta = []
    for p in product:
        if re.search(r'\b%s\b' % p, x):
            ret.extend([re.sub(r'\b%s\b' % p, p2, x) for p2 in product if p != p2])
            ret_meta.extend([(p, p2) for p2 in product if p != p2])
            
    if meta:
        return ret, ret_meta
    else:
        return ret

In [ ]:
data_pos = list(df_beauty[df_beauty["sentiment"] =="positive"]["review_body"].replace(np.nan, '', regex=True))
data_neg = list(df_beauty[df_beauty["sentiment"] =="negative"]["review_body"].replace(np.nan, '', regex=True))

In [ ]:
ret = Perturb.perturb(data_pos, change_product, labels=1, nsamples=2000)


In [ ]:
test = INV(**ret, labels=1)

In [ ]:
test.run(wrapped_pp_beauty)

In [ ]:
test.summary()

In [ ]:
inv_pos_samples = pd.DataFrame({"review_body":dict(ret)["data"]}).sample(n=10,random_state=1)

In [ ]:
ret = Perturb.perturb(data_neg, change_product, labels=0, nsamples=2000)

In [ ]:
test = INV(**ret, labels=0)

In [ ]:
test.run(wrapped_pp_beauty)

In [ ]:
test.summary()

In [ ]:
inv_neg_samples = pd.DataFrame({"review_body":dict(ret)["data"]}).sample(n=10,random_state=1)

## Capability: Temporal Awareness

In [ ]:
suite = TestSuite()

editor.add_lexicon('neg_verb_present', neg_verb_present, overwrite=True)
editor.template('{neg_verb_present}').data

In [ ]:
nouns = ['product','wig','lipstick','cream','oil']
editor.add_lexicon('nouns', nouns, overwrite=True)
pos_adj = ['amazing', 'incredible', 'extraordinary', 'excellent', 'awesome', 'nice']
neg_adj = ['terrible', 'eerie', 'odd', 'awful', 'ugly', 'strange']
editor.add_lexicon('pos_adj', pos_adj, overwrite=True)
editor.add_lexicon('neg_adj', neg_adj, overwrite=True )

In [ ]:
pos_verb_present = ['like', 'enjoy', 'appreciate', 'love',  'recommend', 'admire', 'value', 'welcome']
neg_verb_present = ['hate', 'dislike', 'regret',  'abhor', 'dread', 'despise']
pos_verb_past = ['liked', 'enjoyed', 'appreciated', 'loved', 'admired', 'valued', 'welcomed']
neg_verb_past = ['hated', 'disliked', 'regretted',  'abhorred', 'dreaded', 'despised']

editor.add_lexicon('pos_verb_present', pos_verb_present, overwrite=True)
editor.add_lexicon('neg_verb_present', neg_verb_present, overwrite=True)
editor.add_lexicon('pos_verb_past', pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb_past', neg_verb_past, overwrite=True)
editor.add_lexicon('pos_verb', pos_verb_present+ pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb', neg_verb_present + neg_verb_past, overwrite=True)

In [ ]:
change = ['but', 'even though', 'although', '']
t = editor.template(['I used to think this cream was {neg_adj}, {change} now I think it is {pos_adj}.',
                                 'I think this product is {pos_adj}, {change} I used to think it was {neg_adj}.',
                                 'In the past I thought this lipstick was {neg_adj}, {change} now I think it is {pos_adj}.',
                                 'I think this wig is {pos_adj}, {change} in the past I thought it was {neg_adj}.',
                                ] ,
                                 change=change, unroll=True, nsamples=500, save=True, labels=1)
t += editor.template(['I used to {neg_verb_present} this oil, {change} now I {pos_verb_present} it.',
                                 'I {pos_verb_present} this product, {change} I used to {neg_verb_present} it.',
                                 'In the past I would {neg_verb_present} this shampoo, {change} now I {pos_verb} it.',
                                 'I {pos_verb_present} this cream, {change} in the past I would {neg_verb_present} it.',
                                ] ,
                                change=change, unroll=True, nsamples=500, save=True, labels=1)
t += editor.template(['I used to think this product was {pos_adj}, {change} now I think it is {neg_adj}.',
                                 'I think this cream is {neg_adj}, {change} I used to think it was {pos_adj}.',
                                 'In the past I thought this oil was {pos_adj}, {change} now I think it is {neg_adj}.',
                                 'I think this wig is {neg_adj}, {change} in the past I thought it was {pos_adj}.',
                                ] ,
                                 change=change, unroll=True, nsamples=500, save=True, labels=0)
t += editor.template(['I used to {pos_verb_present} this product, {change} now I {neg_verb_present} it.',
                                 'I {neg_verb_present} this nail polish, {change} I used to {pos_verb_present} it.',
                                 'In the past I would {pos_verb_present} this shampoo, {change} now I {neg_verb_present} it.',
                                 'I {neg_verb_present} this cream, {change} in the past I would {pos_verb_present} it.',
                                ] ,
                                change=change, unroll=True, nsamples=500, save=True, labels=0)

In [ ]:
test = MFT(**t)
description = '''Have two conflicing statements, one about the past and one about the present.
Expect the present to carry the sentiment. Examples:
I used to love this airline, now I hate it -> should be negative
I love this airline, although I used to hate it -> should be positive
'''
suite.add(test, 'used to, but now', 'Temporal', description)

In [ ]:
suite.run(wrapped_pp_beauty, overwrite=True)

In [ ]:
suite.summary()

In [ ]:
temporal_df_beauty = pd.DataFrame({"text":dict(t)["data"],"sentiment":dict(t)["labels"]})

# Hardcases
These are the different cases we're going to include in our tests. These are a mix of different tests, each test descriped below. The aim of these tests is to evaluate the annotators ability to annotate the sentiment of product reviews. 

These tests are made up of both real reviews, sampled from the datasets and synthetic tests, samples which are generated by us.

## Regular samples (20 samples, 50:50 split)
These samples are just normal samples from the dataset. These are intended to be a control test of the annotators generel ability to annotate. If they have a 5% failure rate here, it can be seen as a reflection of their general ability to annotate and should be taken into account when validating their performance on the hard cases.

These tests are real reviews

In [ ]:
reg_pos_samples = df_beauty[df_beauty["sentiment"] == "positive"].sample(n=10, random_state=1)
reg_neg_samples = df_beauty[df_beauty["sentiment"] == "negative"].sample(n=10, random_state=1)

In [ ]:
reg_df = pd.concat((reg_pos_samples[["review_body","sentiment"]], reg_neg_samples[["review_body","sentiment"]]))
reg_df["casetype"] = "regular"

In [ ]:
reg_df

## Invariance product name (20 samples, 50:50 split)
For invariance we want to see if changing domain specific words in the data has an influence on the annotators ability to correnctly annotate sentiment of the review. The change in words should not change the sentiment of the text, but should test if there is a bias towards different domain specific words being used in either negative or positive contexts. As an example, is there a negative connotation or bias towards the word "wig" that might make the model or annotator more inclined to put the review as negative than positive, regardless of the actual sentiment of the text. 

These tests are real reviews which have been slightly modified

In [ ]:
import random

In [ ]:
inv_neg_samples["review_body"] = inv_neg_samples["review_body"].apply(lambda x: random.choice(x))
inv_pos_samples["review_body"] = inv_pos_samples["review_body"].apply(lambda x: random.choice(x))

In [ ]:
inv_pos_samples["sentiment"] = "positive"
inv_neg_samples["sentiment"] = "negative"

In [ ]:
inv_df = pd.concat((inv_pos_samples, inv_neg_samples))
inv_df["casetype"] = "invariance"

In [ ]:
inv_df

## Negation (20 samples, 50:50 split)
Negation is quite a straight forward test. It simply tests the annotators ability to handle double negatives and positive reviews with negative adjectives

These tests are synthetic

In [ ]:
nega_pos_samples = negation_df_beauty[negation_df_beauty["sentiment"]==1].sample(n=10, random_state= 1)
nega_neg_samples = negation_df_beauty[negation_df_beauty["sentiment"]==0].sample(n=10, random_state= 1)

In [ ]:
nega_pos_samples["sentiment"] = "positive"
nega_neg_samples["sentiment"] = "negative"

In [ ]:
nega_df = pd.concat((nega_pos_samples, nega_neg_samples))
nega_df["casetype"] = "negation"

In [ ]:
nega_df.columns = ['review_body', 'sentiment', 'casetype']

In [ ]:
nega_df

## Temporal (20 samples, 50:50 split)
For temporal hard cases, we want to see if participents have a hard time annotating if text has a temporal element to it. In this case, the reviewer describes how they used to have a relationship to the product, but now it's changed "used to, but now..."

The baseline model is not able to handle these kinds of sentences very well, most likely because it goes off of the word counts and not the context of which they are used in the sentence.

These samples are synthetic

In [ ]:
temp_pos_samples = temporal_df_beauty[temporal_df_beauty["sentiment"] == 1].sample(n=10, random_state= 1)
temp_neg_samples = temporal_df_beauty[temporal_df_beauty["sentiment"] == 0].sample(n=10, random_state= 1)

In [ ]:
temp_pos_samples["sentiment"] = "positive"
temp_neg_samples["sentiment"] = "negative"

In [ ]:
temp_df = pd.concat((temp_pos_samples, temp_neg_samples))
temp_df["casetype"] = "temporal"

In [ ]:
temp_df.columns = ['review_body', 'sentiment', 'casetype']

In [ ]:
temp_df

## Fairness, Demographic based (20 samples, 50:50 split)
For fairness we want to see if our participents annotate differently based on the gender of the reviewer, or any minority background the reviewer might have. 

They are also selected based on if the product is targeted at a specific gender or if the creator of the product might be a specific gender. These reviews are selected from the dataset, where these details are overtly expressed, with a few exceptions. 

These exceptions are when a product line's name is mentioned and that product line is targeted at a specific gender or if a creator of a product is a well known person and we know their gender or minority background based on their name being present in the review. As an example, if a review states "This Nicki Minaj album is great!", we can infere that this album is made by a woman of colour.

These tests are sampled from the dataset

In [ ]:
"""
specify which words you're looking for in the string down below in regex format
"""
mask = df_beauty[["review_headline","review_body"]].apply(
    lambda x: x.str.contains(
        "woman of color",
        regex=True
    )
).any(axis=1)

In [ ]:
df_beauty[mask]

Re run this script down below to randomly sample the mask you've created. 

This is done, because you might have generated a lot of data that isn't relevant for the fairness test.

This way you can hand pick samples which you are certain of being correct for the test

In [ ]:
print(len(df_beauty[mask]))
sample = df_beauty[mask].sample(n=1)[["review_body","sentiment"]]
print(sample["review_body"].iloc[0])
print(sample["sentiment"].iloc[0])

In [ ]:
fair_pos_samples = [
    "Turns every bride into a lesbian in front of your very eyes. Very bitchian.",
    "I ordered this, just wondering how effective this sort of product could be. Im a masculine gay guy and can be confused for straight, but I have women tripping over me to talk. A lady tried to pick me up in the GAY grocery store in Wilton Manors one night. She only stopped when I told her I'm gay. Gay men were also more than friendly every time I have worn it. Try it! It works for me!",
    "My MtF trans daughter, loved them! Thank you for making and selling a  great quality product!",
    "I like this wig alot.I got the dark brown one..I am  I am a trans female also..this wig is very feminine which I love ..makes me look very girly after I dress and do all that.I'm very passable.. Only problem is that this wig tangles so fast its dissapointing",
    "Didn't cause any acne on my already blemished skin. The mineral primer and foundation are great. The foundation has a very buildable coverage and the set includes 2 shades. I am an asian and the shade honey fits me well. The setting powder is soso, not worth for a regular size.<br /><br />Honey: neutral undertone<br />Soft Belge: yellow undertone",
    "Loving this serum! The main reason I bought it was to try to revive my eye area. I am a 44 year old female. I've been using it for 2 weeks now and I could see results almost immediately. My puffiness has been reduced, the skin has begun to firm up and looks brighter. I will definitely order this product again!",
    "Very soft and delicate yet scents of a woman not a little girl. Poised!",
    "love. i use it underneath my eyes to set my councilor and on my nose and t zone to avoid shine. I am a woman of color (NW40/ NW45) and it brightens but doesn't ash out or look white. Used in my head shots, perfect powder.",
    "My son had super curly hair and this product has helped us finally control them.  He has autism and this was easy to use and he is able to do it himself.  He loves that he can take care of his hair without mom fussing over him.",
    "Im hispanic and have full lips but when I do this it looks like I got my lips down because it makes them pouty… It doesn't hurt to bad either…"
]

In [ ]:
fair_neg_samples = [
    "All I'll say is, I went to a gay bar with a close female friend of mine. I wore this product, and the guys were all hitting on my female friend and ignoring me. What a waste of money",
    'So upset. As a male to female trans I RELY HEAVILY ON MY WIGS. Got to wear it 3 times. Got it less than a week ago and it got so tangled, and worn, and horrible when I tried to comb it out. I will NEVER buy another one from this company. So so so upset. Also it sheds worse than my dog.',
    "I been using this product for a month now. Commercials say it works 80% +/- on men but when I opened the package and read some info on the product. It works 80% +/- on white men. One problem, I'm not Caucasian. It didn't work for me but it might work for you.",
    "The packaging and name for this product are completely misleading.  I am a woman of color and was expecting this to be a brownish shade, like the packaging.  However, this is a light orangey color that bears absolutely no resemblance to either cocoa or berry when applied.  It is more of a burnt sienna and seems better suited to caucasian skin.",
    "I chose this product because I came off my birth control pills after ten years being on it n needed something to help with the acne break out. I used the whole range of the proactive line n was getting results after three months.  I would not recommend this products or any part of it for black people or people of color who live n very hot climate. The benzoyl peroxide burns your skin even when used n very small amounts. I ended up with  dark skin with black patches on my face. I was a fair skinned person n ended up black n the face while my neck and body was light skinned. The ad really shows mostly white people on it so I should have been warned.",
    "This leaves a white substance on my skin when it dries. It does tighten but it's Not for people of color.",
    "Unfortunately I've had no luck with this particular product. I didn't really notice a difference in the attention I gathered. I DID, however, get great results from the perfume spray for women. Maybe I'm just not around enough gay men? I notice I naturally tend to attract more straight and bisexual men.",
    "I purchased this product back in 1937. Although I expected the architecture of the scent to be more complex it was more reminiscent of Tori Amos' song &#34;In the spring time of his voodoo&#34;. Disappointment. No beaming up qualities but more to align with the gay rights movement. I must admit a quality is as a gay man, my fiance wants to attend more Gay Rights political movements after smelling me but it really gets in the way of our sunday scrabble nights.",
    "I'm hispanic so I'm a little more of the tan skin so my sun spot I was trying to take off is still there with no difference at all.<br />This must work on fair skinned people only?",
    "Being a woman of color, I was highly skeptical on buying a lip stain but was very interested in trying something new... MY CONCLUSION???!!!! Hated it!!! I got this product in desire and victorian and both gave no effect to my lips. Lasting color? NO SIR!!!! The balm was probably the best part of this product even though it breaks in about a week or so if you're not EXTREMELY careful. All in all I am highly dissatisfied...."
]

In [ ]:
fair_pos_df = pd.DataFrame({"review_body":fair_pos_samples})
fair_pos_df["sentiment"] = "positive"

In [ ]:
fair_neg_df = pd.DataFrame({"review_body":fair_neg_samples})
fair_neg_df["sentiment"] = "negative"

In [ ]:
fair_df = pd.concat((fair_pos_df,fair_neg_df))

In [ ]:
fair_df["casetype"] = "fairness"

In [ ]:
fair_df

## Compiling hardcases

In [ ]:
hardcases_beauty = pd.concat((reg_df,inv_df,nega_df,temp_df,fair_df)).reset_index(drop=True)

In [ ]:
hardcases_beauty = hardcases_beauty.sample(frac=1,random_state=1).reset_index(drop=True)

In [ ]:
hardcases_beauty

In [ ]:
with open("hardcases_beauty_cheatsheet.csv", "w+", encoding="utf-8") as file:
    hardcases_beauty.to_csv(file)